In [1]:
import numpy as np
import os
import json, time, copy
import math

from tqdm import tqdm
import random
import pickle
from word2number import w2n
import string, re
from collections import Counter, defaultdict
from pprint import pprint
import spacy
nlp = spacy.load("en_core_web_sm", disable=["ner","textcat","parser"])
np.set_printoptions(precision=4)

In [7]:
txt_dataset = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/txt_dataset_0823_clean_te.json", "r"))

print(Counter([txt_dataset[k]['split'] for k in txt_dataset]))
print(len(set([txt_dataset[k]['Guid'] for k in txt_dataset])))

Counter({'train': 17812, 'test': 4076, 'val': 2455})
24343


In [48]:
file = "Txt_gen_model/Trial1_LargePredictions_4dc3.tsv"
with open(os.path.join("/home/yingshac/CYS/WebQnA/VLP/vlp/light_output/", file), "r") as fp:
    lines = fp.readlines()
    header = lines[0].strip().split('\t')
    rows = lines[1:]
key = dict(zip(header, range(len(header))))
pprint(key)

{'1': 5, 'A': 3, 'Guid': 0, 'Keywords_A': 4, 'Passage': 6, 'Q': 2, 'Qcate': 1}


In [49]:
guid2output = dict([(r.strip().split('\t')[0], r.strip().split('\t')[-1]) for r in rows])
print(len(guid2output))

4180


In [50]:
f = open(os.path.join("/home/yingshac/CYS/WebQnA/VLP/vlp/light_output/", file.split('.')[0]+"_cor.tsv"), "w")
header = ['Guid', 'Qcate', 'Q', 'A', 'Keywords_A', 'Output_conf', 'Output']
v = '{0}\n'.format('\t'.join(map(str, header)))
f.write(v)
for k in txt_dataset:
    if not txt_dataset[k]['split'] == 'test': continue
    guid = txt_dataset[k]['Guid']
    if not guid in guid2output: 
        print(guid)
        continue
    row = [guid, 'text', txt_dataset[k]['Q'], json.dumps(txt_dataset[k]['A']), txt_dataset[k]['Keywords_A'], 1, json.dumps([guid2output[guid]])]
    v = '{0}\n'.format('\t'.join(map(str, row)))
    f.write(v)
f.close()

11a8fd8b-b505-473e-904a-889bd04af07d
7257e221-9e93-45d9-8a81-8728a750f86f
dcebed0b-269c-48b7-90fa-b3dd781e7483


In [21]:
k = random.choice(list(guid2output.keys()))
guid2output[k]

'2-Butoxyethanol is a solvent for paints and surface coatings, as well as cleaning products and inks. It can be obtained in the laboratory by performing a ring opening of 2-propyl-1.'

In [37]:
folder = "/home/yingshac/CYS/WebQnA/VLP/vlp/Human/"
with open(os.path.join(folder, "df_txt_write_all_keep.json"), "r", encoding="utf-8") as file:
    txt_data = json.load(file)
with open(os.path.join(folder, "df_img_write_all_keep.json"), "r", encoding="utf-8") as file:
    mm_data = json.load(file)
print(len(txt_data.values()))
print(len(mm_data.values()))

1000
1000


In [53]:
txt_data['0'].keys()

dict_keys(['Guid', 'Question', 'Qtype', 'AllFactsWithJudgments'])

In [56]:
json.loads(txt_data['0']['AllFactsWithJudgments'])[1]

{'fact': "Tolkien's Middle-earth stories mostly focus on the north-west of the continent. This part of Middle-earth is suggestive of Europe, the north-west of the Old World, with the environs of the Shire intended to be reminiscent of England (more specifically, the West Midlands, with the town at its centre, Hobbiton, at the same latitude as Oxford ).",
 'judgedIsSupporting': True,
 'title': 'Middle-earth - Wikipedia',
 'type': 'supp',
 'url': 'https://en.wikipedia.org/wiki/Middle-earth'}

In [57]:
not json.loads(txt_data['0']['AllFactsWithJudgments'])[1]['type'] == 'supp'

False

In [60]:
compute_retrieval_metrics([0,1,2], [1,3])
F1s = []
REs = []
PRs = []
for k in txt_data:
    datum = txt_data[k]
    pred = []
    gth = []
    i = 0
    for f in json.loads(datum['AllFactsWithJudgments']):
        if f['judgedIsSupporting']: pred.append(i)
        if f['type'] == 'supp': gth.append(i)
        i += 1
    F1, RE, PR = compute_retrieval_metrics(pred, gth)
    F1s.append(F1)
    REs.append(RE)
    PRs.append(PR)
assert len(F1s) == len(REs) == len(PRs)
print(np.mean(F1s))
print(np.mean(REs))
print(np.mean(PRs))

0.9282856195848658
0.9436666666666667
0.9320999999999999


In [92]:
while True:
    k = random.choice(list(mm_data.keys()))
    datum = mm_data[k]
    pred = []
    gth = []
    i = 0
    for f in json.loads(datum['AllFactsWithJudgments']):
        if f['judgedIsSupporting']: pred.append(i)
        if f['type'] == 'supp': gth.append(i)
        i += 1
    F1, RE, PR = compute_retrieval_metrics(pred, gth)
    if F1 < 0.999:
        
        print(pred)
        print(gth)

        print(F1, RE, PR)
        break

[0, 1, 2]
[0, 1]
0.7999952000287999 1.0 0.6666666666666666


In [93]:
compute_retrieval_metrics([0,1,2], [1,3])
F1s = []
REs = []
PRs = []
for k in mm_data:
    datum = mm_data[k]
    pred = []
    gth = []
    i = 0
    for f in json.loads(datum['AllFactsWithJudgments']):
        if f['judgedIsSupporting']: pred.append(i)
        if f['type'] == 'supp': gth.append(i)
        i += 1
    F1, RE, PR = compute_retrieval_metrics(pred, gth)
    F1s.append(F1)
    REs.append(RE)
    PRs.append(PR)
for k in txt_data:
    datum = txt_data[k]
    pred = []
    gth = []
    i = 0
    for f in json.loads(datum['AllFactsWithJudgments']):
        if f['judgedIsSupporting']: pred.append(i)
        if f['type'] == 'supp': gth.append(i)
        i += 1
    F1, RE, PR = compute_retrieval_metrics(pred, gth)
    F1s.append(F1)
    REs.append(RE)
    PRs.append(PR)
assert len(F1s) == len(REs) == len(PRs)
print("F1 = ", np.mean(F1s))
print("RE = ", np.mean(REs))
print("PR = ", np.mean(PRs))

F1 =  0.9054246800241467
RE =  0.9493333333333333
PR =  0.8974273522229403


In [ ]:
compute_retrieval_metrics([0,1,2], [1,3])
F1s = []
REs = []
PRs = []
for k in mm_data:
    datum = mm_data[k]
    pred = []
    gth = []
    i = 0
    for f in json.loads(datum['AllFactsWithJudgments']):
        if f['judgedIsSupporting']: pred.append(i)
        if f['type'] == 'supp': gth.append(i)
        i += 1
    F1, RE, PR = compute_retrieval_metrics(pred, gth)
    F1s.append(F1)
    REs.append(RE)
    PRs.append(PR)
assert len(F1s) == len(REs) == len(PRs)
print(np.mean(F1s))
print(np.mean(REs))
print(np.mean(PRs))

In [59]:
def compute_retrieval_metrics(pred, gth):
    common = len(set(pred).intersection(gth))
    RE = common / len(gth)
    if len(pred) == 0: return 0,0,0
    PR = common / len(pred)
    F1 = 2*PR*RE / (PR + RE + 1e-5)
    return F1, RE, PR